# Preprocessing Exploration

In [1]:
import numpy as np
import pandas as pd
import sqlite3, math
import xgboost as xgb
#from preprocessing.preprocessor import Preprocessor
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
# Load CSV data into Pandas DataFrame 
#train_identity_data = pd.read_csv('../data/raw_data/train_identity.csv') 
train_transaction_data = pd.read_csv('../data/raw_data/train_transaction.csv') 
#test_identity_data = pd.read_csv('../data/raw_data/test_identity.csv') 
test_transaction_data = pd.read_csv('../data/raw_data/test_transaction.csv') 

In [3]:
df = train_transaction_data
df.columns
categorical_cols = ["ProductCD", 
                    "card1", "card2", "card3", "card4", "card5", "card6",
                    "addr1", "addr2",
                    "P_emaildomain", "R_emaildomain",
                    "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9"
                   ]
for col in df[categorical_cols]:
    print(f"{col}: {df[col].nunique()} unique values")


ProductCD: 5 unique values
card1: 13553 unique values
card2: 500 unique values
card3: 114 unique values
card4: 4 unique values
card5: 119 unique values
card6: 4 unique values
addr1: 332 unique values
addr2: 74 unique values
P_emaildomain: 59 unique values
R_emaildomain: 60 unique values
M1: 2 unique values
M2: 2 unique values
M3: 2 unique values
M4: 3 unique values
M5: 2 unique values
M6: 2 unique values
M7: 2 unique values
M8: 2 unique values
M9: 2 unique values


In [4]:
df.card1

0         13926
1          2755
2          4663
3         18132
4          4497
          ...  
590535     6550
590536    10444
590537    12037
590538     7826
590539    15066
Name: card1, Length: 590540, dtype: int64

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class DropHighNaNColumns(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.70):
        self.threshold = threshold
        self.columns_to_keep_ = []

    def fit(self, X, y=None):
        # Store columns where the fraction of NaNs is below the threshold
        self.columns_to_keep_ = X.columns[X.isnull().mean() < self.threshold].tolist()
        return self

    def transform(self, X):
        return X[self.columns_to_keep_].copy()

    def get_feature_names_out(self, input_features=None):
        # Return the final set of columns after dropping
        return self.columns_to_keep_

In [19]:
class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, min_freq=10):
        self.min_freq = min_freq
        self.frequent_categories_ = {}

    def fit(self, X, y=None):
        X_str = X.astype(str)
        for col in X.columns:
            freq = X[col].value_counts()
            self.frequent_categories_[col] = freq[freq >= self.min_freq].index.tolist()
        return self

    def transform(self, X):
        X_str = X.astype(str)
        X_out = X_str.copy()
        for col in X.columns:
            X_out[col] = X_out[col].where(X_out[col].isin(self.frequent_categories_[col]), other='Other')
        self.feature_names_out_ = X_out.columns
        return X_out

    def get_feature_names_out(self, input_features=None):
        return self.feature_names_out_

In [20]:
categorical_cols = ["ProductCD", 
                    "card1", "card2", "card3", "card4", "card5", "card6",
                    "addr1", "addr2",
                    "P_emaildomain", "R_emaildomain",
                    "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9"
                   ]
union_list = list(set(categorical_cols).union({"TransactionID", "isFraud", }))
sub_df = df.drop(columns=union_list)
numeric_cols = sub_df.columns

df.addr2.unique()

for name in categorical_cols:
    if df.dtypes[name] == float:
        df[name] = df[name].astype('Int64')
    

In [21]:
enc = OneHotEncoder(handle_unknown='ignore')

cat_transformer = Pipeline([
    ('drop_nan_cols', DropHighNaNColumns(threshold=0.75)),
    ('rare_class_combine', RareCategoryGrouper(min_freq=3000)),
    ('onehot', enc)
])

num_transformer = Pipeline([
    ('drop_nan_cols', DropHighNaNColumns(threshold=0.75)),
#    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('cat', cat_transformer, categorical_cols),
    ('num', num_transformer, numeric_cols)
])

In [22]:
df.addr1

0         315
1         325
2         330
3         476
4         420
         ... 
590535    272
590536    204
590537    231
590538    387
590539    299
Name: addr1, Length: 590540, dtype: Int64

In [25]:
df_clean = preprocessor.fit_transform(df)
df_clean.shape

(590540, 263)

In [26]:
preprocessor.get_feature_names_out()

array(['cat__ProductCD_C', 'cat__ProductCD_H', 'cat__ProductCD_R',
       'cat__ProductCD_S', 'cat__ProductCD_W', 'cat__card1_Other',
       'cat__card2_Other', 'cat__card3_Other', 'cat__card4_Other',
       'cat__card4_american express', 'cat__card4_discover',
       'cat__card4_mastercard', 'cat__card4_visa', 'cat__card5_Other',
       'cat__card6_Other', 'cat__card6_credit', 'cat__card6_debit',
       'cat__addr1_Other', 'cat__addr2_Other', 'cat__P_emaildomain_Other',
       'cat__P_emaildomain_anonymous.com', 'cat__P_emaildomain_aol.com',
       'cat__P_emaildomain_att.net', 'cat__P_emaildomain_comcast.net',
       'cat__P_emaildomain_gmail.com', 'cat__P_emaildomain_hotmail.com',
       'cat__P_emaildomain_icloud.com', 'cat__P_emaildomain_live.com',
       'cat__P_emaildomain_msn.com', 'cat__P_emaildomain_outlook.com',
       'cat__P_emaildomain_yahoo.com', 'cat__M1_Other', 'cat__M1_T',
       'cat__M2_F', 'cat__M2_Other', 'cat__M2_T', 'cat__M3_F',
       'cat__M3_Other', 'cat__M3_

In [27]:
df_clean.shape

(590540, 263)

In [62]:
train_transaction_data.P_emaildomain.unique()
mask = train_transaction_data.P_emaildomain == 'gmail'
train_transaction_data.loc[mask, 'P_emaildomain'] = 'gmail.com'

df = train_transaction_data[0:50]
df_valid = train_transaction_data[200:250]
df_test = test_transaction_data[0:50]
mask = train_transaction_data.isFraud == 1
dfF = train_transaction_data[mask]
df = pd.concat([df,dfF])
df.reset_index()
df = df[0:60]
df
df_valid

train_transaction_data.P_emaildomain.unique()

array([nan, 'gmail.com', 'outlook.com', 'yahoo.com', 'mail.com',
       'anonymous.com', 'hotmail.com', 'verizon.net', 'aol.com', 'me.com',
       'comcast.net', 'optonline.net', 'cox.net', 'charter.net',
       'rocketmail.com', 'prodigy.net.mx', 'embarqmail.com', 'icloud.com',
       'live.com.mx', 'live.com', 'att.net', 'juno.com', 'ymail.com',
       'sbcglobal.net', 'bellsouth.net', 'msn.com', 'q.com',
       'yahoo.com.mx', 'centurylink.net', 'servicios-ta.com',
       'earthlink.net', 'hotmail.es', 'cfl.rr.com', 'roadrunner.com',
       'netzero.net', 'gmx.de', 'suddenlink.net', 'frontiernet.net',
       'windstream.net', 'frontier.com', 'outlook.es', 'mac.com',
       'netzero.com', 'aim.com', 'web.de', 'twc.com', 'cableone.net',
       'yahoo.fr', 'yahoo.de', 'yahoo.es', 'sc.rr.com', 'ptd.net',
       'live.fr', 'yahoo.co.uk', 'hotmail.fr', 'hotmail.de',
       'hotmail.co.uk', 'protonmail.com', 'yahoo.co.jp'], dtype=object)

In [24]:
prep = Preprocessor(df, drop_method={"missing":70}, drop_list=[])
prep.run()
#df = prep.df
dropped_cols = prep.dropped_cols
new_columns = prep.cols

> /home/db/gitProjects/fraud-detection/src/preprocessing/preprocessor.py(43)one_hot()
     41             current_col = self.df[name].to_frame()
     42             pdb.set_trace()
---> 43             nan_mask = self.df[name].isna()
     44             cats = self.df[name].unique()
     45             #encoder = OneHotEncoder(categories=cats.tolist(), handle_unknown='ignore')



ipdb>  c


> /home/db/gitProjects/fraud-detection/src/preprocessing/preprocessor.py(43)one_hot()
     41             current_col = self.df[name].to_frame()
     42             pdb.set_trace()
---> 43             nan_mask = self.df[name].isna()
     44             cats = self.df[name].unique()
     45             #encoder = OneHotEncoder(categories=cats.tolist(), handle_unknown='ignore')



ipdb>  c


> /home/db/gitProjects/fraud-detection/src/preprocessing/preprocessor.py(43)one_hot()
     41             current_col = self.df[name].to_frame()
     42             pdb.set_trace()
---> 43             nan_mask = self.df[name].isna()
     44             cats = self.df[name].unique()
     45             #encoder = OneHotEncoder(categories=cats.tolist(), handle_unknown='ignore')



...

ipdb>  
ipdb>  c


> /home/db/gitProjects/fraud-detection/src/preprocessing/preprocessor.py(43)one_hot()
     41             current_col = self.df[name].to_frame()
     42             pdb.set_trace()
---> 43             nan_mask = self.df[name].isna()
     44             cats = self.df[name].unique()
     45             #encoder = OneHotEncoder(categories=cats.tolist(), handle_unknown='ignore')



ipdb>  c


> /home/db/gitProjects/fraud-detection/src/preprocessing/preprocessor.py(43)one_hot()
     41             current_col = self.df[name].to_frame()
     42             pdb.set_trace()
---> 43             nan_mask = self.df[name].isna()
     44             cats = self.df[name].unique()
     45             #encoder = OneHotEncoder(categories=cats.tolist(), handle_unknown='ignore')



ipdb>  c


In [26]:
new_columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320',
       'V321'],
      dtype='object', length=335)

In [45]:
X = df[new_columns]
X_cat = df.select_dtypes(include='object')
cat_cols = X_cat.columns
enc = OneHotEncoder(drop="if_binary", handle_unknown="ignore")
enc.fit(X_cat)
X_new = enc.transform(X_cat)

X_test = df_valid[cat_cols]

In [46]:
enc.categories_

[array(['C', 'H', 'R', 'S', 'W'], dtype=object),
 array(['american express', 'discover', 'mastercard', 'visa'], dtype=object),
 array(['credit', 'debit'], dtype=object),
 array(['anonymous.com', 'aol.com', 'gmail.com', 'hotmail.com', 'mail.com',
        'me.com', 'outlook.com', 'verizon.net', 'yahoo.com', nan],
       dtype=object),
 array(['anonymous.com', 'gmail.com', 'hotmail.com', 'outlook.com', nan],
       dtype=object),
 array(['T', nan], dtype=object),
 array(['F', 'T', nan], dtype=object),
 array(['F', 'T', nan], dtype=object),
 array(['M0', 'M1', 'M2', nan], dtype=object),
 array(['F', 'T', nan], dtype=object),
 array(['F', 'T', nan], dtype=object),
 array(['F', 'T', nan], dtype=object),
 array(['F', 'T', nan], dtype=object),
 array(['F', 'T', nan], dtype=object)]

In [54]:
X_test_new = enc.transform(X_test)
X_test_new.toarray()
X_test
X_test.P_emaildomain.unique()

/home/db/gitProjects/fraud-detection/venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array(['gmail.com', 'aol.com', 'hotmail.com', 'yahoo.com',
       'anonymous.com', 'icloud.com', nan, 'gmail'], dtype=object)

In [55]:
X_cat.P_emaildomain.unique()

array([nan, 'gmail.com', 'outlook.com', 'yahoo.com', 'mail.com',
       'anonymous.com', 'hotmail.com', 'verizon.net', 'aol.com', 'me.com'],
      dtype=object)

In [8]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)

In [9]:
X_over

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,...,P_emaildomain_verizon.net,P_emaildomain_yahoo.com,M1_T,M2_T,M3_T,M4_M0,M4_M1,M4_M2,M5_T,M6_T
0,2987000,86400,68.500,13926,NaN,150.0,142.0,315.0,87.0,19.0,...,NaN,NaN,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1,2987001,86401,29.000,2755,404.0,150.0,102.0,325.0,87.0,NaN,...,0.0,0.0,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0
2,2987002,86469,59.000,4663,490.0,150.0,166.0,330.0,87.0,287.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,2987003,86499,50.000,18132,567.0,150.0,117.0,476.0,87.0,NaN,...,0.0,1.0,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0
4,2987004,86506,50.000,4497,514.0,150.0,102.0,420.0,87.0,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2987288,90986,155.521,16578,545.0,185.0,226.0,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN
96,2987367,92350,225.000,4425,562.0,150.0,197.0,472.0,87.0,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,2987203,89760,445.000,18268,583.0,150.0,226.0,251.0,87.0,NaN,...,0.0,0.0,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0
98,2987405,92999,90.570,4504,500.0,185.0,219.0,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN


In [10]:
y_over

0     0
1     0
2     0
3     0
4     0
     ..
95    1
96    1
97    1
98    1
99    1
Name: isFraud, Length: 100, dtype: int64

In [11]:
model = xgb.XGBClassifier()
model.fit(X_over, y_over)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [37]:
y_pred = model.predict(X_over)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_over, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 100.00%


In [38]:
prep_valid = Preprocessor(df_valid, drop_list = dropped_cols)
prep_valid.drop()
df_valid_new = prep_valid.df
#df_test[df.columns]

In [39]:
df_valid_new.columns
#df_test_new

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)

In [40]:
df.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)

In [44]:
cols_valid = set(df_valid_new.columns.to_list())
cols_train = set(df.columns.to_list())

In [45]:
cols_train.difference(cols_valid)

set()